In [183]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_curve
import os
import re

In [208]:
scraped_papers = pd.read_csv("data/scraped_papers_final.csv")
scraped_papers

,Unnamed: 0,DOI,URL,year,title,publisher,text
0,0,10.1002/9781119578826.ch15,https://doi.org/10.1002/9781119578826.ch15,2024,['Quantum Dots Solar Cells'],Wiley,Paper #: 7386_1\nTitle: Role of carrier deloca...
1,1,10.1002/9781119578826.ch16,https://doi.org/10.1002/9781119578826.ch16,2024,['Singlet Fission for Solar Cells'],Wiley,Paper #: 7742_1\nTitle: Perovskite-Based Tande...
2,2,10.1002/9781119578826.ch27,https://doi.org/10.1002/9781119578826.ch27,2024,['Deployment of<scp>Solar</scp>Photovoltaic Sy...,Wiley,Paper #: 9299_1\nTitle: Inhomogeneous Halide A...
3,3,10.1002/9781119578826.ch6,https://doi.org/10.1002/9781119578826.ch6,2024,['Tunnel Oxide Passivated Contact (<scp>TOPCon...,Wiley,Paper #: 7949_1\nTitle: Efficiency assessment ...
4,5,10.1002/9781119600862.ch3,https://doi.org/10.1002/9781119600862.ch3,2024,['Solar Thermochemical Fuels'],Wiley,Paper #: 7743_1\nTitle: Manufacturing Cost Ana...
...,...,...,...,...,...,...,...
9873,10259,10.7836/kses.2023.43.5.043,https://doi.org/10.7836/kses.2023.43.5.043,2023,['Performance Evaluation of LSTM Based Solar I...,The Korean Solar Energy Society,Paper #: 16412_1\nTitle: Supporting Informatio...
9874,10260,10.7836/kses.2023.43.6.013,https://doi.org/10.7836/kses.2023.43.6.013,2024,['A Methodology for Calculating the Limit Capa...,The Korean Solar Energy Society,Paper #: 16980_1\nTitle: Nanosecond Laser Pass...
9875,10261,10.7836/kses.2023.43.6.067,https://doi.org/10.7836/kses.2023.43.6.067,2024,['Prediction Model For Colored BIPV Modules'],The Korean Solar Energy Society,Paper #: 19568_1\nTitle: Improve Perovskite So...
9876,10262,10.7836/kses.2023.43.6.087,https://doi.org/10.7836/kses.2023.43.6.087,2024,['Transformer-Based Photovoltaic Power Predict...,The Korean Solar Energy Society,Paper #: 19781_1\nTitle: Electron-hole diffusi...


In [188]:
def passivator_frequency(text):
    passivator_names = ["passivation", "passivator", "passivating"]
    words = text.lower().split()
    return sum([words.count(name) for name in passivator_names])

In [197]:
def keyword_classification(text):
    passivators = ["passivation", "passivator", "passivating"]
    pces = ["pce", "power conversion efficiency"]
    return any(keyword in text.lower() for keyword in passivators) and any(keyword in text.lower() for keyword in pces) and passivator_frequency(text) > 2

In [198]:
scraped_papers["label"] = scraped_papers["text"].apply(keyword_classification)

C:\Users\nicco\AppData\Local\Temp\ipykernel_39204\1994295596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scraped_papers["label"] = scraped_papers["text"].apply(keyword_classification)


In [199]:
relevant_papers = scraped_papers[scraped_papers["label"] == 1]
relevant_papers.shape[0]

1378

In [140]:
relevant_papers.to_csv("data/relevant_papers.csv", index=False)

### Generating Samples for Spot-Checking

In [ ]:
folder_path = "data/relevant_samples_final"

sampled_df = relevant_papers.sample(n=15, random_state=41)

for i, text in enumerate(sampled_df["text"]):
    file_path = os.path.join(folder_path, f"sample_{i}.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)